In [35]:
# imports
import nest_asyncio
import qdrant_client

from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex, ServiceContext, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate

from llama_index.core.postprocessor import SentenceTransformerRerank

from IPython.display import Markdown, display



In [ ]:

nest_asyncio.apply()

In [18]:
# configs
## qdrant
collection_name = "chat_with_docs"

# docs
directory = "./docs"

EMBED_MODEL = "BAAI/bge-large-en-v1.5"


In [4]:
client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)


In [9]:
reader = SimpleDirectoryReader(input_dir=directory, required_exts=[".pdf"], recursive=True)

In [10]:
data = reader.load_data()

In [14]:
# type(data)
len(data)

32

In [15]:
data[0]

Document(id_='5b2046f5-24b0-4653-8c97-990605dbf2fa', embedding=None, metadata={'page_label': '1', 'file_name': 'dspy.pdf', 'file_path': '/Users/prakharpatidar/Documents/github/rag-sytems/docs/dspy.pdf', 'file_type': 'application/pdf', 'file_size': 460814, 'creation_date': '2025-05-26', 'last_modified_date': '2025-05-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Preprint\nDSP Y: C OMPILING DECLARATIVE LANGUAGE\nMODEL CALLS INTO SELF -IMPROVING PIPELINES\nOmar Khattab,1 Arnav Singhvi,2\nParidhi Maheshwari,4 Zhiyuan Zhang,1\nKeshav Santhanam,1 Sri Vardhamanan,6 Saiful Haq,6\nAshutosh Sharma,6 Thomas T. Joshi,7 Hanna Moazam,8\nHeather 

In [17]:
def create_index(documents):

    vector_store = QdrantVectorStore(client=client,
                                     collection_name=collection_name)
    
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    index = VectorStoreIndex.from_documents(documents,
                                            storage_context=storage_context)
    
    return index

In [21]:
embed_model = HuggingFaceEmbedding(model_name=EMBED_MODEL,
                                   trust_remote_code=True)
Settings.embed_model = embed_model

In [22]:
index = create_index(data)

In [25]:
llm = Ollama(model="llama3.2", request_timeout=120.0)


In [26]:
Settings.llm = llm

In [29]:
template = """Context information is below:
              ---------------------
              {context_str}
              ---------------------
              Given the context information above I want you to think
              step by step to answer the query in a crisp manner,
              incase you don't know the answer say 'I don't know!'
            
              Query: {query_str}
        
              Answer:"""

qa_prompt_tmpl = PromptTemplate(template)

In [31]:
rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", 
    top_n=3
)

In [32]:
query_engine = index.as_query_engine(similarity_top_k=10,
                                     node_postprocessors=[rerank])
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)


In [33]:
response = query_engine.query("What exactly is DSPy?")

In [36]:
display(Markdown(str(response)))

Based on the provided context, DSPy (Declarative Signature-based Prompting for eXplainability) is a programming model that translates prompting techniques into parameterized declarative modules. It allows users to define "signatures" - typed declarations of functions that specify what a text transformation needs to do, rather than how a specific LM should be prompted to implement that behavior.